In [2]:
import numpy as np
import pandas as pd
import h5py
import pysam
import os
from modisco.visualization import viz_sequence
from modisco import util
from matplotlib import pyplot as plt
import pybedtools
from bs4 import BeautifulSoup

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [3]:
dttype = 'ATAC'
mode = "counts"
celltype="GM12878"
modisco_path = '/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/ATAC/'+celltype+'/merge_folds_new_may_05_24/'+mode+'/modisco_old_format.h5'
ppm_dir = ''

htmld="/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/"+dttype+'/'+celltype+"/merge_folds_new_may_05_24/"+mode+"/motifs.html"
tomtom = pd.read_html(htmld)


background=[0.25, 0.25, 0.25, 0.25]

In [4]:
def trim_motif_new(cwm, motif, trim_threshold=0.20):
    """
    Given the PFM and motif (both L x 4 arrays) (the motif could be the
    PFM itself), trims `motif` by cutting off flanks of low information
    content in `pfm`. `min_ic` is the minimum required information
    content. If specified this trimmed motif will be extended on either
    side by `pad` bases.
    If no base passes the `min_ic` threshold, then no trimming is done.
    """
    
    score = np.sum(np.abs(cwm), axis=1)
    trim_thresh = np.max(score) * trim_threshold  # Cut off anything less than 30% of max score
    pass_inds = np.where(score >= trim_thresh)[0]
    trimmed = motif[np.min(pass_inds): np.max(pass_inds) + 1]
 
    if not trimmed.size:
        return motif
    
    return trimmed

def import_tfmodisco_motifs(tfm_results_path, trim=True, only_pos=True):
    """
    Imports the PFMs to into a dictionary, mapping `(x, y)` to the PFM,
    where `x` is the metacluster index and `y` is the pattern index.
    Arguments:
        `tfm_results_path`: path to HDF5 containing TF-MoDISco results
        `out_dir`: where to save motifs
        `trim`: if True, trim the motif flanks based on information content
        `only_pos`: if True, only return motifs with positive contributions
    Returns the dictionary of PFMs.
    """ 
    pfms = {}
    with h5py.File(tfm_results_path, "r") as f:
        metaclusters = f["metacluster_idx_to_submetacluster_results"]
        num_metaclusters = len(metaclusters.keys())
        for metacluster_i, metacluster_key in enumerate(metaclusters.keys()):
            metacluster = metaclusters[metacluster_key]
            if "patterns" not in metacluster["seqlets_to_patterns_result"]:
                continue
            patterns = metacluster["seqlets_to_patterns_result"]["patterns"]
            num_patterns = len(patterns["all_pattern_names"][:])
            for pattern_i, pattern_name in enumerate(patterns["all_pattern_names"][:]):
#                pattern_name = pattern_name.decode()
                pattern_name = pattern_name

                pattern = patterns[pattern_name]
                pfm = pattern["sequence"]["fwd"][:]
                cwm = pattern["task0_contrib_scores"]["fwd"][:]
                
                # Check that the contribution scores are overall positive
                if only_pos and np.sum(cwm) < 0:
                    continue
                    
                if trim:
                    pfm = trim_motif_new(cwm, cwm)
                else:
                    pfm = cwm
                    
                pfms["%d_%d" % (metacluster_i,pattern_i)] = pfm
    return pfms

In [5]:
pfms = import_tfmodisco_motifs(modisco_path, trim=False)

In [6]:
for key in pfms:
    f = open(os.path.join(ppm_dir,mode+"_"+key+".pfm"),"w")
    #print(pfms[key])
    np.savetxt(f, pfms[key], fmt='%f')
    f.close()
    

In [7]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,pos_patterns.pattern_0,33945,NaN,NaN,IRF1_MOUSE.H11MO.0.A,1.138220e-03,NaN,IRF1_HUMAN.H11MO.0.A,1.138220e-03,NaN,STAT2_HUMAN.H11MO.0.A,1.138220e-03,NaN
1,pos_patterns.pattern_1,31055,NaN,NaN,CTCF_MA0139.1,7.873460e-14,NaN,CTCF_HUMAN.H11MO.0.A,1.302790e-09,NaN,CTCF_MOUSE.H11MO.0.A,1.827970e-08,NaN
2,pos_patterns.pattern_2,20226,NaN,NaN,SPI1_HUMAN.H11MO.0.A,3.365930e-04,NaN,SPIB_MOUSE.H11MO.0.A,3.365930e-04,NaN,SPI1_MOUSE.H11MO.0.A,3.365930e-04,NaN
3,pos_patterns.pattern_3,17141,NaN,NaN,RUNX1_HUMAN.H11MO.0.A,3.627260e-02,NaN,RUNX3_HUMAN.H11MO.0.A,1.073110e-01,NaN,RUNX1_MOUSE.H11MO.0.A,1.073110e-01,NaN
4,pos_patterns.pattern_4,16150,NaN,NaN,JDP2_MA0655.1,3.099440e-03,NaN,JDP2_bZIP_3,3.099440e-03,NaN,FOS+JUND_MA1141.1,3.099440e-03,NaN
5,pos_patterns.pattern_5,12502,NaN,NaN,NFKB1_HUMAN.H11MO.1.B,6.058370e-05,NaN,NFKB1_MOUSE.H11MO.0.A,6.058370e-05,NaN,TF65_HUMAN.H11MO.0.A,6.058370e-05,NaN
6,pos_patterns.pattern_6,10067,NaN,NaN,IRF4_HUMAN.H11MO.0.A,2.708130e-07,NaN,IRF8_HUMAN.H11MO.0.B,2.359790e-06,NaN,IRF8_MOUSE.H11MO.0.A,2.359790e-06,NaN
7,pos_patterns.pattern_7,6890,NaN,NaN,Gabpa_MA0062.2,2.058070e-06,NaN,ELK4_MA0076.2,1.053440e-05,NaN,ELK4_HUMAN.H11MO.0.A,1.053440e-05,NaN
8,pos_patterns.pattern_8,6841,NaN,NaN,KLF12_HUMAN.H11MO.0.C,9.140990e-05,NaN,SP1_HUMAN.H11MO.0.A,9.140990e-05,NaN,SP3_HUMAN.H11MO.0.B,3.909220e-04,NaN
9,pos_patterns.pattern_9,3674,NaN,NaN,NFYB_HUMAN.H11MO.0.A,5.336540e-03,NaN,NFYB_MOUSE.H11MO.0.A,5.336540e-03,NaN,NFYA_HUMAN.H11MO.0.A,6.019980e-03,NaN


In [8]:

tomtom[0]["pattern"] = tomtom[0]["pattern"].str.replace("pos_patterns.pattern","0").str.replace("neg_patterns.pattern","1")


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [9]:
tomtom[0]

,pattern,num_seqlets,modisco_cwm_fwd,modisco_cwm_rev,match0,qval0,match0_logo,match1,qval1,match1_logo,match2,qval2,match2_logo
0,0_0,33945,NaN,NaN,IRF1_MOUSE.H11MO.0.A,1.138220e-03,NaN,IRF1_HUMAN.H11MO.0.A,1.138220e-03,NaN,STAT2_HUMAN.H11MO.0.A,1.138220e-03,NaN
1,0_1,31055,NaN,NaN,CTCF_MA0139.1,7.873460e-14,NaN,CTCF_HUMAN.H11MO.0.A,1.302790e-09,NaN,CTCF_MOUSE.H11MO.0.A,1.827970e-08,NaN
2,0_2,20226,NaN,NaN,SPI1_HUMAN.H11MO.0.A,3.365930e-04,NaN,SPIB_MOUSE.H11MO.0.A,3.365930e-04,NaN,SPI1_MOUSE.H11MO.0.A,3.365930e-04,NaN
3,0_3,17141,NaN,NaN,RUNX1_HUMAN.H11MO.0.A,3.627260e-02,NaN,RUNX3_HUMAN.H11MO.0.A,1.073110e-01,NaN,RUNX1_MOUSE.H11MO.0.A,1.073110e-01,NaN
4,0_4,16150,NaN,NaN,JDP2_MA0655.1,3.099440e-03,NaN,JDP2_bZIP_3,3.099440e-03,NaN,FOS+JUND_MA1141.1,3.099440e-03,NaN
5,0_5,12502,NaN,NaN,NFKB1_HUMAN.H11MO.1.B,6.058370e-05,NaN,NFKB1_MOUSE.H11MO.0.A,6.058370e-05,NaN,TF65_HUMAN.H11MO.0.A,6.058370e-05,NaN
6,0_6,10067,NaN,NaN,IRF4_HUMAN.H11MO.0.A,2.708130e-07,NaN,IRF8_HUMAN.H11MO.0.B,2.359790e-06,NaN,IRF8_MOUSE.H11MO.0.A,2.359790e-06,NaN
7,0_7,6890,NaN,NaN,Gabpa_MA0062.2,2.058070e-06,NaN,ELK4_MA0076.2,1.053440e-05,NaN,ELK4_HUMAN.H11MO.0.A,1.053440e-05,NaN
8,0_8,6841,NaN,NaN,KLF12_HUMAN.H11MO.0.C,9.140990e-05,NaN,SP1_HUMAN.H11MO.0.A,9.140990e-05,NaN,SP3_HUMAN.H11MO.0.B,3.909220e-04,NaN
9,0_9,3674,NaN,NaN,NFYB_HUMAN.H11MO.0.A,5.336540e-03,NaN,NFYB_MOUSE.H11MO.0.A,5.336540e-03,NaN,NFYA_HUMAN.H11MO.0.A,6.019980e-03,NaN


In [10]:
tomtom[0][["pattern","num_seqlets"]].to_csv(os.path.join(ppm_dir,mode+"_counts.csv"),sep=",",index=False, header=False)